# Assignment 2: Monte Carlo (MC) Methods in Finance

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import seaborn as sns
import pandas as pd


## Part I Option Valuation

### Q1: Monte Carlo for European Call？ Options

#### Convergence Study

In [ ]:
# Parameters
S0 = 100       # initial stock price
K = 99         # strike price
T = 1.0        # time to maturity
r = 0.06       # risk-free rate
sigma = 0.20   # volatility
M_trials = [10000, 50000, 100000, 500000, 1000000,2000000, 3000000, 4000000, 5000000]  # different numbers of trials

option_prices = []
std_errors = []

# Monte Carlo simulation to estimate European Option price
def monte_carlo_simulation(S0, K, T, r, sigma, M):
    dt = T / M  # time step
    discount_factor = np.exp(-r * T)  # discount factor for present value
    
    # Generate M stock price paths
    Z = np.random.standard_normal(M)
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z) 
    payoffs = np.maximum(ST - K, 0)
    option_price = discount_factor * np.mean(payoffs)  # average discounted payoff
    std_error = discount_factor * np.std(payoffs) / np.sqrt(M)
    
    return option_price, std_error

for M in M_trials:
    price, error = monte_carlo_simulation(S0, K, T, r, sigma, M)
    option_prices.append(price)
    std_errors.append(error)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

# Option Price Convergence
ax1.plot(M_trials, option_prices, marker='o')
ax1.set_xlabel('Number of Trials', fontsize=14)
ax1.set_ylabel('Option Price', fontsize=14)
ax1.tick_params(labelsize=12)
ax1.grid(True)

# Standard Error Convergence
ax2.plot(M_trials, std_errors, marker='o', color='red')
ax2.set_xlabel('Number of Trials', fontsize=14)
ax2.set_ylabel('Standard Error', fontsize=14)
ax2.tick_params(labelsize=12)
ax2.grid(True)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
print(option_prices)

#### Comparison with Assignment 1 Binomial Model
- Option Price Estimates: Assignment 1 provided a binomial tree model price of approximately 11.55 for a European Call option with specific parameters (1-year maturity, strike price of 99, risk-free rate of 6%, current stock price of 100, and volatility of 20%)​​. The Monte Carlo simulation generated option prices at different trial numbers as printed. When M = 50000, the option price calculated is about 11.54.
- Convergence Analysis: The binomial tree model showed convergence of option prices as the number of steps increased​. In Monte Carlo simulation, the standard error converges as the number of trials increases, showing increasing accuracy of the Monte Carlo estimate.
- Impact of Volatility: 

#### Varying Volality and Strike Price

In [ ]:
S0 = 100
T = 1.0 
r = 0.06
M = 100000  

strike_prices = [80, 90, 100, 110, 120,150]
volatilities = [0.1,0.2,0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Store the results
results = []

# Perform simulations for varying strike prices and volatilities
for K in strike_prices:
    for sigma in volatilities:
        price, error = monte_carlo_simulation(S0, K, T, r, sigma, M)
        results.append((K, sigma, price, error))

df = pd.DataFrame(results, columns=['Strike Price', 'Volatility', 'Option Price', 'Standard Error'])

pivot_price = df.pivot(index="Strike Price", columns="Volatility", values="Option Price")
pivot_error = df.pivot(index="Strike Price", columns="Volatility", values="Standard Error")

# Plotting heatmaps for Option Price and Standard Error
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)

sns.heatmap(pivot_price, annot=True, fmt=".2f", linewidths=0.5)
plt.xlabel("Volatility", fontsize=14)
plt.ylabel("Strike Price", fontsize=14)
plt.title("Option Price (European Call)", fontsize=16)

plt.subplot(1, 2, 2)
sns.heatmap(pivot_error, annot=True, fmt=".3f", linewidths=0.5)
plt.title("Standard Error", fontsize=16)
plt.xlabel("Volatility", fontsize=14)
plt.ylabel("Strike Price", fontsize=14)

plt.tight_layout()
plt.show()



## Part II Estimation of Sensitivities in MC

### Q1: Use bump-and-revalue method to estimate delta

In [ ]:
# Bump-and-revalue method to estimate Delta
def bump_and_revalue(S0, K, T, r, sigma, N, seed=np.random.seed(), bump_size=0.01):
    # Monte Carlo simulation to estimate the option price for original and bumped S0
    def simulate_option_price(S0, r, sigma, T, N, n_simulations=10000):
        dt = T / N
        prices = np.zeros(n_simulations)
        for i in range(n_simulations):
            # Simulate the asset price path
            dW = np.random.normal(scale=np.sqrt(dt), size=N)
            path = S0 * np.cumprod(np.exp((r - 0.5 * sigma ** 2) * dt + sigma * dW))
            # Calculate the payoff for a European call option
            payoff = max(path[-1] - K, 0)
            prices[i] = payoff
        # Discount the average payoff to get the option price
        option_price = np.exp(-r * T) * np.mean(prices)
        return option_price

    # Calculate the option price for unbumped S0
    np.random.seed(seed)
    price_unbumped = simulate_option_price(S0, r, sigma, T, N)

    # Bump the underlying asset price
    S0_bumped = S0 + S0 * bump_size

    # Calculate the option price for bumped S0 with the same random seed
    np.random.seed(seed)  # Use the same seed for reproducibility
    price_bumped_same_seed = simulate_option_price(S0_bumped, r, sigma, T, N)

    # Reset the seed and calculate the option price for bumped S0 with a different seed
    np.random.seed()  # Use a different seed
    price_bumped_diff_seed = simulate_option_price(S0_bumped, r, sigma, T, N)

    # Estimate Delta using the same seed
    delta_same_seed = (price_bumped_same_seed - price_unbumped) / (S0 * bump_size)

    # Estimate Delta using a different seed
    delta_diff_seed = (price_bumped_diff_seed - price_unbumped) / (S0 * bump_size)

    return delta_same_seed, delta_diff_seed

# Given parameters
S0 = 100 
T = 1.0 
N = 200 
K = 99 
r = 0.06 
sigma = 0.2 
seeds = np.arange(101)

deltas_same_seed = []
deltas_diff_seed = []
for seed in seeds:
    # Calculate Delta with the bump-and-revalue method
    delta_same_seed, delta_diff_seed = bump_and_revalue(S0, K, T, r, sigma, N, seed)
    deltas_same_seed.append(delta_same_seed)
    deltas_diff_seed.append(delta_diff_seed)    

In [ ]:
# Data adjust
var_same = format(np.var(deltas_same_seed), ".1e")
var_diff = format(np.var(deltas_diff_seed), ".1e")
# Box-plot
data = {f'Same seed(mean={round(np.mean(deltas_same_seed),2)},var={var_same})': deltas_same_seed, 
        f'Different seed(mean={round(np.mean(deltas_diff_seed),2)},var={var_diff})': deltas_diff_seed}
data_long_format = pd.DataFrame(list(data.items()), columns=['Seed type', 'Delta value']).explode('Delta value')
data_long_format['Delta value'] = data_long_format['Delta value'].astype(float)

plt.figure(figsize=(8, 6))
sns.boxplot(x='Seed type', y='Delta value', data=data_long_format)
plt.title('Comparison of using same or different seed (Bumpsize = 0.01)', fontsize = 16)
plt.xlabel('Seed type', fontsize = 15)
plt.ylabel('Delta value', fontsize = 15)
plt.grid(True)
plt.show()

print("Result under 100 simulation round")
print(f"For Same seed : mean = {np.mean(deltas_same_seed)}, variance = {np.var(deltas_same_seed)}")
print(f"For Different seed : mean = {np.mean(deltas_diff_seed)}, variance = {np.var(deltas_diff_seed)}")

Experiment with Different Bump Sizes

Try different `bump sizes` and compare the results in `AS1`.

Results in AS1:
- With the same parameter settings as in `AS1`, we obtain **Delta**: `0.674`


In [ ]:
bumpsizes = [0.03, 0.01, 0.003, 0.001]
S0 = 100 
T = 1.0 
N = 200 
K = 99 
r = 0.06 
sigma = 0.2 
seeds = np.arange(101)

for bumpsize in bumpsizes:
    deltas_same_seed = []
    deltas_diff_seed = []
    for seed in seeds:
        # Calculate Delta with the bump-and-revalue method
        delta_same_seed, delta_diff_seed = bump_and_revalue(S0, K, T, r, sigma, N, seed, bumpsize)
        deltas_same_seed.append(delta_same_seed)
        deltas_diff_seed.append(delta_diff_seed)  
    # Box-plot
    var_same = format(np.var(deltas_same_seed), ".1e")
    var_diff = format(np.var(deltas_diff_seed), ".1e")
    data = {f'Same seed(mean={round(np.mean(deltas_same_seed),3)},var={var_same})': deltas_same_seed, 
            f'Different seed(mean={round(np.mean(deltas_diff_seed),3)},var={var_diff})': deltas_diff_seed}
    data_long_format = pd.DataFrame(list(data.items()), columns=['Seed type', 'Delta value']).explode('Delta value')
    data_long_format['Delta value'] = data_long_format['Delta value'].astype(float)

    plt.figure(figsize=(8, 6))
    sns.boxplot(x='Seed type', y='Delta value', data=data_long_format)
    plt.title(f'Comparison of using same or different seed (Bumpsize = {bumpsize})', fontsize = 16)
    plt.xlabel('Seed type', fontsize = 15)
    plt.ylabel('Delta value', fontsize = 15)
    plt.grid(True)
    plt.show()  

### Q2: Use multiple methods to compute delta for a digital option 

Bump-and-revalue method

In [ ]:
# Bump-and-revalue method to estimate Delta
def bump_and_revalue(S0, K, T, r, sigma, N, seed=np.random.seed(), bump_size=0.001):
    # Monte Carlo simulation to estimate the option price for original and bumped S0
    def simulate_option_price(S0, r, sigma, T, N, n_simulations=10000):
        dt = T / N
        prices = np.zeros(n_simulations)
        for i in range(n_simulations):
            # Simulate the asset price path
            dW = np.random.normal(scale=np.sqrt(dt), size=N)
            path = S0 * np.cumprod(np.exp((r - 0.5 * sigma ** 2) * dt + sigma * dW))
            # Calculate the payoff for a European call option
            payoff = 1 if path[-1] - K > 0 else 0
            prices[i] = payoff
        # Discount the average payoff to get the option price
        option_price = np.exp(-r * T) * np.mean(prices)
        return option_price

    # Calculate the option price for unbumped S0
    np.random.seed(seed)
    price_unbumped = simulate_option_price(S0, r, sigma, T, N)

    # Bump the underlying asset price
    S0_bumped = S0 + S0 * bump_size

    # Calculate the option price for bumped S0 with the same random seed
    np.random.seed(seed)  # Use the same seed for reproducibility
    price_bumped_same_seed = simulate_option_price(S0_bumped, r, sigma, T, N)

    # Reset the seed and calculate the option price for bumped S0 with a different seed
    np.random.seed()  # Use a different seed
    price_bumped_diff_seed = simulate_option_price(S0_bumped, r, sigma, T, N)

    # Estimate Delta using the same seed
    #print(price_bumped_same_seed,price_unbumped)
    delta_same_seed = (price_bumped_same_seed - price_unbumped) / (S0 * bump_size)

    # Estimate Delta using a different seed
    delta_diff_seed = (price_bumped_diff_seed - price_unbumped) / (S0 * bump_size)

    return delta_same_seed, delta_diff_seed

# Given parameters
S0 = 100
T = 1.0 
N = 200 
K = 100
r = 0.06 
sigma = 0.2 
seeds = np.arange(101)

deltas_same_seed = []
deltas_diff_seed = []
for seed in seeds:
    # Calculate Delta with the bump-and-revalue method
    delta_same_seed, delta_diff_seed = bump_and_revalue(S0, K, T, r, sigma, N, seed)
    deltas_same_seed.append(delta_same_seed)
    deltas_diff_seed.append(delta_diff_seed)    



In [ ]:
# Data adjust
var_same = format(np.var(deltas_same_seed), ".1e")
var_diff = format(np.var(deltas_diff_seed), ".1e")
# Box-plot
data = {f'Same seed': deltas_same_seed, 
        f'Different seed': deltas_diff_seed}
#data = {f'Same seed(mean={round(np.mean(deltas_same_seed),3)},var={var_same})': deltas_same_seed, 
#        f'Different seed(mean={round(np.mean(deltas_diff_seed),3)},var={var_diff})': deltas_diff_seed}
data_long_format = pd.DataFrame(list(data.items()), columns=['Seed type', 'Delta value']).explode('Delta value')
data_long_format['Delta value'] = data_long_format['Delta value'].astype(float)

plt.figure(figsize=(8, 6))
sns.boxplot(x='Seed type', y='Delta value', data=data_long_format)
plt.title('Comparison of using same or different seed (Bumpsize = 0.01)', fontsize = 16)
plt.xlabel('Seed type', fontsize = 15)
plt.ylabel('Delta value', fontsize = 15)
plt.grid(True)
plt.show()

print("Result under 100 simulation round")
print(f"For Same seed : mean = {round(np.mean(deltas_same_seed),5)}, variance = {var_same}")
print(f"For Different seed : mean = {round(np.mean(deltas_diff_seed),5)}, variance = {var_diff}")

Pathwise method

$H(S_T) = 
\begin{cases}
1 & \text{if } S_T > K, \\
0 & \text{if } S_T \leq K.
\end{cases}$


A smoothed approximation to the digital option payoff can
be:

$H_\epsilon(S_T) = \frac{1}{1 + \exp\left(-\frac{S_T - K}{\epsilon}\right)}$

Compute partial derivatives of smoothed payoff functions:

$\frac{\partial H_\epsilon(S_T)}{\partial S_T} = \frac{\exp\left(-\frac{S_T - K}{\epsilon}\right)}{\epsilon \left(1 + \exp\left(-\frac{S_T - K}{\epsilon}\right)\right)^2}$


In [ ]:
# Parameter setting
S0 = 100    
K = 100     
T = 1.0     
r = 0.06    
sigma = 0.2 
M = 10000   
epsilons = [0.1, 0.3, 1]  # smooth approximation parameter

# Partial derivatives of smoothed payoff functions
def logistic_derivative(ST, K, epsilon):
    e_term = np.exp(-(ST - K) / epsilon)
    return e_term / (epsilon * (1 + e_term)**2)

delta_estimates_dif_epsilon = []
simulation_round = 100 # Simulation rounds
for epsilon in epsilons:
    delta_estimates = []
    for _ in range(simulation_round):
        np.random.seed()
        Z = np.random.normal(0, 1, M)
        ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)

        logistic_derivatives = logistic_derivative(ST, K, epsilon)

        adjusted_derivatives = logistic_derivatives * ST / S0

        delta_estimate = np.exp(-r * T) * np.mean(adjusted_derivatives)
        delta_estimates.append(delta_estimate)
    delta_estimates_dif_epsilon.append(delta_estimates)

In [ ]:
# Data adjust
means = []
vars = []
for i in range(len(delta_estimates_dif_epsilon)):
    vars.append(format(np.var(delta_estimates_dif_epsilon[i]), ".1e"))
    means.append(round(np.mean(delta_estimates_dif_epsilon[i]), 5))

# Draw box-plot
data = {}
for i in range(len(delta_estimates_dif_epsilon)):
    #key = f'Epsilon: {epsilons[i]} (mean={means[i]}, var={vars[i]})'
    key = f'Epsilon: {epsilons[i]}'
    data[key] = delta_estimates_dif_epsilon[i]

data_long_format = pd.DataFrame(list(data.items()), columns=['Epsilons', 'Delta value']).explode('Delta value')
data_long_format['Delta value'] = data_long_format['Delta value'].astype(float)

plt.figure(figsize=(8, 6))
sns.boxplot(x='Epsilons', y='Delta value', data=data_long_format)
plt.title('Comparison of Different Epsilons', fontsize = 16)
plt.xlabel('Epsilons', fontsize = 15)
plt.ylabel('Delta value', fontsize = 15)
plt.grid(True)
plt.show()

print("Result under 100 simulation round")
for i in range(len(delta_estimates_dif_epsilon)):
    print(f"For Epsilon = {epsilons[i]} : mean = {means[i]}, variance = {vars[i]}")

Likelihood Ratio Method

We estimate the derivative using following method:


$\begin{aligned}
\frac{\partial\pi_0(p)}{\partial p}& =\int_0^\infty H(x)\frac{\partial g(x,p)}{\partial p}dx  \\
&=\int_0^\infty H(x)\frac{\partial\log(g(x,p))}{\partial p}g(x,p)dx \\
&=\mathbb{E}_{\mathbb{Q}}\left[H(S_T)\frac{\partial\log(g(S_T,p))}{\partial p}\right]
\end{aligned}$

In this particular question, we have:

$\hat{\Delta}=e^{-rT}\frac1M\sum_{i=1}^M\left(1_{\{S_{T,i}>K\}}\frac{Z_i}{S_0\sigma\sqrt{T}}\right)$

In [ ]:
S0 = 100    
K = 100     
T = 1.0     
r = 0.06    
sigma = 0.2 
M = 10000   
simulation_round = 100

LRM_deltas = []
for _ in range(simulation_round):
    np.random.seed()
    Z = np.random.normal(0, 1, M)
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
    # 1_{ST>K} is an indicator function. When ST>K, it takes 1, otherwise it takes 0.
    indicator = (ST > K).astype(int)
    LRM_delta = np.exp(-r * T) * np.mean(indicator * Z / (S0 * sigma * np.sqrt(T)))
    LRM_deltas.append(LRM_delta)

print(np.mean(LRM_deltas))
print(np.var(LRM_deltas))

In [ ]:
# Data adjust

var=format(np.var(LRM_deltas), ".1e")
mean=round(np.mean(LRM_deltas), 5)

# Draw box-plot
data = {f'LRM Delta': LRM_deltas,}
data_long_format = pd.DataFrame(list(data.items()), columns=['LRM method', 'Delta value']).explode('Delta value')
data_long_format['Delta value'] = data_long_format['Delta value'].astype(float)

plt.figure(figsize=(8, 6))
sns.boxplot(x='LRM method', y='Delta value', data=data_long_format)
plt.title('Result of LRM method', fontsize = 16)
plt.xlabel('')
plt.ylabel('Delta value', fontsize = 15)
plt.grid(True)
plt.show()

print("Result under 100 simulation round")
print(f"For LRM method : mean = {mean}, variance = {var}")

Result comparison

In [ ]:
datas = []
data_bump_same = {"name":"Bump-and-revalue method with same seed",
                  "mean": round(np.mean(deltas_same_seed),5),
                  "var":var_same}
data_bump_diff = {"name":"Bump-and-revalue method with diff seed",
                  "mean": round(np.mean(deltas_diff_seed),5),
                  "var":var_diff}
datas.append(data_bump_same)
datas.append(data_bump_diff)
for i in range(3):
    data_pathwise = {"name":f"Pathwise method with epsilon = {epsilons[i]}",
                  "mean": means[i],
                  "var":vars[i]}
    datas.append(data_pathwise)

data_LRM = {"name":"LRM method",
            "mean": mean,
            "var":var}
datas.append(data_LRM)

# Convert data to DataFrame
df = pd.DataFrame(datas)

pd.set_option('display.float_format', '{:.4f}'.format) 

print(df)

## Part III Variance Reduction

### Q1: Compare the Asian call option result between the analytical expression and Monte-Carlo simulations

In [ ]:
S0 = 100 
K = 100   
T = 1      
r = 0.05     
sigma = 0.2
N = 252   

def geometric_asian_call_price(S0, K, T, r, sigma, N):
    # Adjust volatility and interest rates based on the characteristics of Asian options
    sigma_hat = sigma * np.sqrt((2 * N + 1) / (6 * (N + 1)))
    r_hat = (r - 0.5 * sigma**2 + sigma_hat**2) / 2

    # Calculate d1 and d2
    d1 = (np.log(S0 / K) + (r_hat + 0.5 * sigma_hat**2) * T) / (sigma_hat * np.sqrt(T))
    d2 = d1 - sigma_hat * np.sqrt(T)

    # Calculate Asian call option prices based on the BS model
    geometric_asian_call_theoretical = S0 * np.exp((r_hat - r) * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return geometric_asian_call_theoretical

geometric_asian_call_theoretical = geometric_asian_call_price(S0, K, T, r, sigma, N)
print(geometric_asian_call_theoretical)


In [ ]:
# Use Monte-Carlo simulation to compute Asian call option
# Simulate Brownian motion path
def simulate_gbm(S0, r, sigma, T, n_steps):
    dt = T/n_steps  
    dW = np.random.normal(scale=np.sqrt(dt), size=n_steps)
    log_S = np.log(S0) + np.cumsum((r - 0.5 * sigma**2) * dt + sigma * dW)
    S = np.exp(log_S)  
    return S

def asian_call_payoff(S_path, K, r, T):
    geometric_mean = np.exp(np.mean(np.log(S_path)))  
    payoff = np.maximum(geometric_mean - K, 0)        
    discounted_payoff = np.exp(-r * T) * payoff     
    return discounted_payoff

In [ ]:
S0 = 100       
K = 100        
T = 1          
r = 0.05       
sigma = 0.2
N = 10000      
n_simulations = 10000  # Number of Monte Carlo simulations

payoffs = np.zeros(n_simulations)

for i in range(n_simulations):
    S_path = simulate_gbm(S0, r, sigma, T, N)
    payoffs[i] = asian_call_payoff(S_path, K, r, T)

# Calculate the average payoff and hence the price of the option
Asian_call_Monte_Carlo = np.mean(payoffs)
geometric_asian_call_theoretical = geometric_asian_call_price(S0, K, T, r, sigma, N)

print(f"Asian_call_Monte_Carlo = {Asian_call_Monte_Carlo}, Simulation time = {n_simulations}")
print(f"Asian_call_Analytical_result = {geometric_asian_call_theoretical}")
print(f"Relative error = {abs(Asian_call_Monte_Carlo - geometric_asian_call_theoretical)/geometric_asian_call_theoretical}")

In [ ]:
# Simulation parameters
S0 = 100
K = 100
T = 1
r = 0.05
sigma = 0.2
N = 10000
n_simulations = 10000
n_batches = 30  # Number of batches to simulate

# Run Monte Carlo simulations in batches
batch_results = np.zeros(n_batches)
theoretical_price = geometric_asian_call_price(S0, K, T, r, sigma, N)

for batch in range(n_batches):
    np.random.seed()  # Ensure randomness for each batch
    payoffs = np.zeros(n_simulations)
    for i in range(n_simulations):
        S_path = simulate_gbm(S0, r, sigma, T, N)
        payoffs[i] = asian_call_payoff(S_path, K, r, T)
    batch_results[batch] = np.mean(payoffs)



In [ ]:
# Draw box-plot
plt.figure(figsize=(8, 6))
sns.boxplot(data=batch_results, color="skyblue")
plt.axhline(y=theoretical_price, color='r', linestyle='--', label='Theoretical Price')
plt.title('Monte Carlo Simulations of Asian Call Option Price',fontsize = 15)
plt.ylabel('Option Price',fontsize = 14)
plt.xticks([0], ['Asian Call Price'],fontsize = 14)
plt.legend()
plt.show()


### Q2 : Explain how the strategy of using this as a control variate works.

Control Variates Method

The **Control Variates Method** is a method used to reduce variance in Monte Carlo simulations. The core idea of this method is to use a control variable with a known expected value to reduce the variance of the simulation output. Here we use the value of an Asian option based on geometric averages as a control variable to reduce variance.

Specific Implementation Steps:

1. **Identify a Control Variate**: Select the geometric average Asian option as the control variate for the arithmetic average Asian option.

2. **Compute the Expected Value of the Control Variate**:  Calculate the expected value of the geometric average Asian option.

3. **Simulate Both Payoffs**: During the simulation, compute both the payoff of the option of interest (arithmetic average Asian option) and the payoff of the control variate (geometric average Asian option).

4. **Calculate the Covariance and Variance**: Estimate the covariance between the payoff and the control variate, and the variance of the control variate, from the simulated data.

5. **Adjust the Payoff**: Use the covariance and the known expected value of the control variate to adjust the payoff of the option of interest. The adjustment is typically made by constructing a linear combination of the payoff and the control variate. The coefficients of this linear combination are chosen to minimize the variance of the estimator.

6. **Estimate the Option Price**: Estimate the Option Price: Calculate the price of the option of interest using the adjusted payoffs.


### Q3：Apply the control variates technique

In [ ]:
geometric_asian_call_theoretical = geometric_asian_call_price(S0, K, T, r, sigma, N)

# Adding calculation of geometric mean
def asian_call_payoff_modified(S_path, K, r, T):
    arithmetic_mean = np.mean(S_path)
    geometric_mean = np.exp(np.mean(np.log(S_path)))
    payoff_arithmetic = np.maximum(arithmetic_mean - K, 0)
    payoff_geometric = np.maximum(geometric_mean - K, 0)
    discounted_payoff_arithmetic = np.exp(-r * T) * payoff_arithmetic
    discounted_payoff_geometric = np.exp(-r * T) * payoff_geometric
    return discounted_payoff_arithmetic, discounted_payoff_geometric

def control_variates_simulation(S0, K, T, r, sigma, N):
    payoffs_arithmetic = np.zeros(n_simulations)
    payoffs_geometric = np.zeros(n_simulations)

    # Simulation
    for i in range(n_simulations):
        S_path = simulate_gbm(S0, r, sigma, T, N)
        payoff_arithmetic, payoff_geometric = asian_call_payoff_modified(S_path, K, r, T)
        payoffs_arithmetic[i] = payoff_arithmetic
        payoffs_geometric[i] = payoff_geometric

    # Calculate the adjustment amount of the control variable
    cov_matrix = np.cov(payoffs_arithmetic, payoffs_geometric)
    cov_XY = cov_matrix[0, 1]
    var_Y = cov_matrix[1, 1]
    c = cov_XY / var_Y

    # Adjusting the simulated value of the arithmetic mean Asian call option using control variables
    adjusted_payoffs = payoffs_arithmetic - c * (payoffs_geometric - geometric_asian_call_theoretical)
    adjusted_price_estimate = np.mean(adjusted_payoffs)

    return np.mean(payoffs_arithmetic), adjusted_price_estimate

S0 = 100       
K = 100        
T = 1          
r = 0.05       
sigma = 0.2    
N = 10000      
n_simulations = 10000 # Number of Monte Carlo simulations

Basic_estimate, adjusted_estimate =  control_variates_simulation(S0, K, T, r, sigma, N)
print("Basic Arithmetic Mean Price Estimate:", Basic_estimate)
print("Adjusted Arithmetic Mean Price Estimate:", adjusted_estimate)

In [ ]:
# Run simulation 100 times
results_arithmetic = []
results_adjusted = []
simulation_time_cv = 100
for _ in range(simulation_time_cv):
    mean_arithmetic, adjusted_estimate = control_variates_simulation(S0, K, T, r, sigma, N)
    results_arithmetic.append(mean_arithmetic)
    results_adjusted.append(adjusted_estimate)

In [ ]:
# Box-plot
data = {'Basic Monte Carlo Method': results_arithmetic, 'Control Variates': results_adjusted}
data_long_format = pd.DataFrame(list(data.items()), columns=['Simulation method', 'Price Estimate']).explode('Price Estimate')
data_long_format['Price Estimate'] = data_long_format['Price Estimate'].astype(float)

plt.figure(figsize=(8, 6))
sns.boxplot(x='Simulation method', y='Price Estimate', data=data_long_format)
plt.title('Comparison of estimate result of two models', fontsize = 16)
plt.xlabel('Simulation method', fontsize = 15)
plt.ylabel('Price Estimate', fontsize = 15)
plt.grid(True)
plt.show()

print("Result under 100 simulation round")
print(f"For Basic Monte Carlo Method : mean = {np.mean(results_arithmetic)}, variance = {np.var(results_arithmetic)}")
print(f"For Control Variates Method : mean = {np.mean(results_adjusted)}, variance = {np.var(results_adjusted)}")